# [NTDS'18] milestone 2: network models
[ntds'18]: https://github.com/mdeff/ntds_2018

[Hermina Petric Maretic](https://people.epfl.ch/hermina.petricmaretic), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `<your team number>`
* Students: `<the name of all students in the team>`
* Dataset: `<the dataset you used to complete the milestone>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* In the first part, you cannot import any other library than we imported. In the second part, you are allowed to import any library you want.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The purpose of this milestone is to explore various random network models, analyse their properties and compare them to your network. In the first part of the milestone you will implement two random graph models and try to fit them to your network. In this part you are not allowed to use any additional package. In the second part of the milestone you will choose a third random graph model that you think shares some properties with your network. You will be allowed to use additional packages to construct this network, but you must explain your network choice. Finally, make your code as clean as possible, and keep your textual answers short.

## Part 0

Import the adjacency matrix of your graph that you constructed in milestone 1, as well as the number of nodes and edges of your network.

In [ ]:
import numpy as np

adjacency =  np.load('adjacency_undirected.npz')['arr_0'] # the adjacency matrix
n_nodes =  adjacency.shape[0] # the number of nodes in the network
n_edges =  int(np.sum(adjacency)/2) # the number of edges in the network

## Part 1

**For the computation of this part of the milestone you are only allowed to use the packages that have been imported in the cell below.**

In [ ]:
%matplotlib inline

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Question 1

Create a function that constructs an Erdős–Rényi graph.

In [ ]:
def erdos_renyi(n, proba, seed=None):
    """Create an instance from the Erdos-Renyi graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    p: float
        Edge probability. A number between 0 and 1.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    if seed is not None:
        np.random.seed(seed)
        
    adjacency=np.zeros((n,n))
    
    # todo: make more efficient
    #adjacency = np.random.choice(2, [n, n], p=[1-proba, proba])
    for i in range(n):
        for j in range(i+1,n): 
            adjacency[i,j]=np.random.choice([0,1],p=[1-proba, proba])
            adjacency[j,i]=adjacency[i,j]
    
    return adjacency

In [ ]:
er = erdos_renyi(5, 0.6, 9765)
plt.spy(er)
plt.title('Erdos-Renyi (5, 0.6)')

In [ ]:
er = erdos_renyi(10, 0.4, 7648)
plt.spy(er)
plt.title('Erdos-Renyi (10, 0.4)')

### Question 2

Use the function to create a random Erdos-Renyi graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
proba= n_edges/(n_nodes*(n_nodes-1)/2)
randomER= erdos_renyi(n_nodes,proba)
plt.spy(randomER)

**Your answer here.**
We chose the same number of nodes as our graph. In order to have a similar number of edges as our graph, we chose the probability to be the number of edges divided by the maximum number of edges possible for a graph on n_nodes

### Question 3

Create a function that constructs a Barabási-Albert graph.

In [ ]:
def barabasi_albert(n, m, m0=2, seed=None):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    m0: int (optional)
        Number of nodes for the inital connected network.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    assert m <= m0
    
    if seed is not None:
        np.random.seed(seed)
    
    adjacency = np.zeros([n, n], dtype=int)
    degree = np.zeros(n, dtype=int)

    # generate initial connected network with one edge per added node. (m0-1 edges)
    for i in range(1, m0):
        target = np.random.choice(i, 1)
        adjacency[i, target] = adjacency[target, i] = 1
        degree[i] += 1
        degree[target] += 1

    # Grow network
    for i in range(m0, n):
        # Preferential attachment: probability that the new node connects to node i 
        dist = degree[:i] / np.sum(degree[:i])

        # Choose m links without replacement with given probability distribution
        targets = np.random.choice(i, m, replace=False, p=dist)
        adjacency[i,targets] = adjacency[targets, i] = 1
        degree[i] += 1
        degree[targets] += 1
 
    return adjacency

In [ ]:
ba = barabasi_albert(5, 1, 2, 9087)
plt.spy(ba)
plt.title('Barabasi-Albert (5, 1)')

In [ ]:
ba = barabasi_albert(10, 2, 3, 8708)
plt.spy(ba)
plt.title('Barabasi-Albert (10, 2)')

### Question 4

Use the function to create a random Barabási-Albert graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
m0 = 24 # choose nb hubs?
m = int((n_edges - m0 + 1) / (n_nodes - m0))
randomBA = barabasi_albert(n_nodes, m, m0, 8708)
plt.spy(randomBA)
plt.title('Barabasi-Albert ({}, {}, {})'.format(n_nodes, m, m0))

**Your answer here**

### Question 5

Compare the number of edges in all three networks (your real network, the Erdős–Rényi network, and the Barabási-Albert netowk).

In [ ]:
n_ER = np.sum(randomER)/2
n_BA = np.sum(randomBA)/2
n_wiki = n_edges

print(n_ER, n_BA, n_wiki)

### Question 6

Implement a function that computes the [Kullback–Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between two probability distributions.
We'll use it to compare the degree distributions of networks.

In [ ]:
def kl_divergence(p, q):
    """Compute the KL divergence between probability distributions of degrees of two networks.
    
    Parameters
    ----------
    p: np.array
        Probability distribution of degrees of the 1st graph.
    q: np.array
        Probability distribution of degrees of the 2nd graph.
    Note: the two arrays should have same length
    Returns
    -------
    kl
        The KL divergence between the two distributions.
    """
    assert p.size == q.size , 'distributions should be of the same length '
    # first step is ignoring the degrees that have 0 in either distribution
    idx_nonzero_p = np.nonzero(p)
    p = p[idx_nonzero_p]
    q = q[idx_nonzero_p]
    idx_nonzero_q = np.nonzero(q)
    p = p[idx_nonzero_q]
    q = q[idx_nonzero_q]
    
    # now we can make the dot product, taking advantage of numpy fast vector operations
    kl= np.dot(p, np.log(p/q))
    return kl

In [ ]:
p_test = np.array([0.2, 0.2, 0.2, 0.4])
q_test = np.array([0.3, 0.3, 0.1, 0.3])
kl_divergence(p_test, q_test)

In [ ]:
# same result as wiki page
p_test = np.array([0.36,0.48,0.16])
q_test = np.array([0.333,0.333,0.333])
kl_divergence( q_test, p_test)

In [ ]:
# introduce 0
kl_divergence(np.array([0, 1]), np.array([1, 0]))

In [ ]:
# same distrib
kl_divergence(np.array([0, 1]), np.array([0,1]))

### Question 7: 

Compare the degree distribution of your network to each of the two synthetic ones, in terms of KL divergence.

**Hint:** Make sure you normalise your degree distributions to make them valid probability distributions.

**Hint:** Make sure none of the graphs have disconnected nodes, as KL divergence will not be defined in that case. If that happens with one of the randomly generated networks, you can regenerate it and keep the seed that gives you no disconnected nodes.

In [ ]:
#degree distribution of our network 
degree_wiki=np.sum(adjacency, axis=0)
degree_distribution_wiki= np.histogram(degree_wiki,np.arange(max(degree_wiki)+1))[0]/n_nodes
#compute degree distribution Erdos Reny Graph
degree_ER=np.sum(randomER, axis =0)
degree_distribution_ER= np.histogram(degree_ER,np.arange(max(degree_ER)+1))[0]/n_nodes
#degree distribution AB

In [ ]:
#theoretical degeree distribution in ER graph
degree_distribution_ER_theoretical=np.zeros(n_nodes)
for k in range(n_nodes):
    degree_distribution_ER_theoretical[k]= scipy.special.binom(n_nodes,k)*np.power(proba,k)*np.power(1-proba,n_nodes-1-k)


### Question 8

Plot the degree distribution historgrams for all three networks. Are they consistent with the KL divergence results? Explain.

In [ ]:
# Your code here.

**Your answer here.**

### Question 9

Imagine you got equal degree distributions. Would that guarantee you got the same graph? Explain.

**Your answer here.**

## Part 2

**You are allowed to use any additional library here (e.g., NetworkX, PyGSP, etc.).** Be careful not to include something here and use it in part 1!

### Question 10

Choose a random network model that fits you network well. Explain your choice. 

**Hint:** Check lecture notes for different network models and their properties. Your choice should be made based on at least one property you'd expect to be similar.

**Your answer here.**

### Question 11

Explain (in short) how the chosen model works.

**Your answer here.**

### Question 12

Create a random graph from that model, such that the number of nodes is the same as in your graph.

In [ ]:
# Your code here.

### Question 13

Check the properties you expected to be similar, and compare to your network.

In [ ]:
# Your code here.

Are the results what you expected? Explain.

**Your answer here.**